In [84]:
import numpy as np, tensorflow as tf
from functools import partial
import matplotlib.pyplot as plt
import pandas as pd, os
import tensorflow as tf, pydicom as dicom
from sklearn.model_selection import train_test_split

In [85]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
#GCS_PATH = KaggleDatasets().get_gcs_path()
TRAIN_IMG_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images'
#BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512, 512]
EPOCHS = 25
TARGET_COLS  = [
        "bowel_injury", "extravasation_injury",
        "kidney_healthy", "kidney_low", "kidney_high",
        "liver_healthy", "liver_low", "liver_high",
        "spleen_healthy", "spleen_low", "spleen_high",
    ]

In [86]:
train = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv')
train = train.head(50)
train['patient_id'] = train['patient_id'].astype(str)
X_train, X_valid, y_train, y_valid = train_test_split(train.patient_id, train.drop(['patient_id'], axis = 1))
y_train['patient_id'] = X_train
y_valid['patient_id'] = X_valid
train = y_train
valid = y_valid
display(train)

,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury,patient_id
31,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10550
43,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10759
2,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10007
9,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10124
20,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1027
8,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10105
16,1,0,0,1,1,0,0,0,1,0,0,0,1,1,10217
11,1,0,1,0,1,0,0,1,0,0,0,0,1,1,10132
7,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10104
38,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10696


In [87]:
import tensorflow as tf
import numpy as np

def image_to_bytes(patient_id):
    for directory in os.listdir(os.path.join(TRAIN_IMG_PATH, patient_id)):
        files = os.listdir(os.path.join(TRAIN_IMG_PATH, patient_id, directory))
        files.sort(key = lambda x: int(x.split('.')[0]))
        files = files[7::int(len(files)//7)]
        files = [os.path.join(TRAIN_IMG_PATH, patient_id, directory, x) for x in files]
        img_arrays = []
        for img in files:
            ds = dicom.dcmread(img)
            img_arrays.append(ds.pixel_array)
        img_3d = np.array(img_arrays)
    return img_3d  #.tobytes()

def create_example(patient_id, labels):
    image_data = image_to_bytes(patient_id)
    #print(np.shape(image_data))
    # Assuming you have a NumPy array with int16 data
    #image_data = np.array([1, 2, 3], dtype=np.int16)
    #image_data = image_to_bytes('10275')
    #print(image_data)
    #print('image data')

    # Convert the tensor to a compatible data type (tf.int32 in this case)
    image_data_compat = tf.constant(image_data, dtype=tf.int32)

    # Serialize the tensor
    serialized_tensor = tf.io.serialize_tensor(image_data_compat)
    #print(tf.shape(serialized_tensor))
    """
    # Deserialize the tensor
    restored_tensor = tf.io.parse_tensor(serialized_tensor, out_type=tf.int32)

    # Convert the restored tensor back to the original data type (tf.int16)
    restored_tensor = tf.cast(restored_tensor, dtype=tf.int16)

    # Convert the restored tensor to a NumPy array
    restored_array = restored_tensor.numpy()

    print(restored_array)
    print('restored_array')
    """
    feature = {
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_tensor.numpy()])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=labels)),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

def write_tfrecords(dataset, output_path):
    with tf.io.TFRecordWriter(output_path) as writer:
        for index, row in dataset.iterrows():
            # Create a TFRecord example and write it to a TFRecord file
            example = create_example(row['patient_id'], row[TARGET_COLS].values)
            writer.write(example.SerializeToString())
            
train_path = "train.tfrecords"
write_tfrecords(train, train_path)
valid_path = "valid.tfrecords"
write_tfrecords(valid, valid_path)


In [ ]:
def parse_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),  # Change to BytesList
        'label': tf.io.VarLenFeature(tf.int64),  # Change to Int64List
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    data = parsed['image']  # This will be a bytes tensor
    label = tf.sparse.to_dense(parsed['label'], default_value=0)  # Convert sparse to dense tensor
    return data, label

def load_tfrecords_dataset(tfrecords_filenames, batch_size=32, shuffle_buffer_size=1000):
    dataset = tf.data.TFRecordDataset(tfrecords_filenames)
    dataset = dataset.map(parse_example)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    return dataset

tfrecords_filenames = ['/kaggle/working/train.tfrecords']
batch_size = 32
shuffle_buffer_size = 1000
dataset = load_tfrecords_dataset(tfrecords_filenames, batch_size, shuffle_buffer_size)

#image_batch, label_batch = next(iter(dataset))
#print(type(image_batch))
for batch in dataset:
    image_batch, label_batch = batch
    break
    # Process the data_batch and label_batch

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10, 10))
    print(image_batch)
    # Convert to a NumPy array using tf.make_ndarray()
    #numpy_array = tf.make_ndarray(image_batch)
    #print(numpy_array)
    # Deserialize the tensor
    restored_tensor = tf.io.parse_tensor(image_batch, out_type=tf.int32)

    # Convert the restored tensor back to the original data type (tf.int16)
    restored_tensor = tf.cast(restored_tensor, dtype=tf.int16)

    # Convert the restored tensor to a NumPy array
    restored_array = restored_tensor.numpy()

    print(restored_array)
    print(tf.shape(image_batch))
    for n in range(25):
        ax = plt.subplot(5, 5, n + 1)
        plt.imshow(image_batch[n] / 255.0)
        plt.title("BENIGN")
        plt.axis("off")


show_batch(image_batch.numpy(), label_batch.numpy())


In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "label": tf.io.FixedLenFeature([], tf.int64),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled:
        label = tf.cast(example["label"], tf.int32)
        return image, label
    return image

In [ ]:
def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

In [ ]:
def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:

"""def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=4)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

train_dataset = get_dataset('/kaggle/input/rsna-make-tfrec/train.tfrecords')
valid_dataset = get_dataset("/kaggle/input/rsna-make-tfrec/valid.tfrecords")
#test_dataset = get_dataset(TEST_FILENAMES, labeled=False)
"""
image_batch, label_batch = next(iter(dataset))


def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10, 10))
    for n in range(25):
        ax = plt.subplot(5, 5, n + 1)
        plt.imshow(image_batch[n] / 255.0)
        if label_batch[n]:
            plt.title("MALIGNANT")
        else:
            plt.title("BENIGN")
        plt.axis("off")


show_batch(image_batch.numpy(), label_batch.numpy())

In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "melanoma_model.h5", save_best_only=True
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

In [ ]:
def make_model():
    base_model = tf.keras.applications.Xception(
        input_shape=(*IMAGE_SIZE, 3), include_top=False, weights="imagenet"
    )

    base_model.trainable = False

    inputs = tf.keras.layers.Input([*IMAGE_SIZE, 3])
    x = tf.keras.applications.xception.preprocess_input(inputs)
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(8, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.7)(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss="binary_crossentropy",
        metrics=tf.keras.metrics.AUC(name="auc"),
    )

    return model

In [ ]:
with strategy.scope():
    model = make_model()

history = model.fit(
    train_dataset,
    epochs=2,
    validation_data=valid_dataset,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

In [ ]:
def show_batch_predictions(image_batch):
    plt.figure(figsize=(10, 10))
    for n in range(25):
        ax = plt.subplot(5, 5, n + 1)
        plt.imshow(image_batch[n] / 255.0)
        img_array = tf.expand_dims(image_batch[n], axis=0)
        plt.title(model.predict(img_array)[0])
        plt.axis("off")


image_batch = next(iter(test_dataset))

show_batch_predictions(image_batch)

# Credits 
https://keras.io/examples/keras_recipes/tfrecord/